In [34]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [35]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [36]:
# WebClient instantiates a client that can call API methods
# When using Bolt, you can use either `app.client` or the `client` passed to listeners.
client = WebClient(token=os.environ.get("SLACK_BOT_TOKEN"))

ここまで必要↑

In [12]:
channel_id = "C04UNBSBRJT" # test

In [8]:
try:
    # Call the conversations.list method using the WebClient
    result = client.chat_postMessage(
        channel=channel_id,
        text="Hello world!"
        # You could also use a blocks[] array to send richer content
    )
    # Print result, which includes information about the message (like TS)
    # print(result)

except SlackApiError as e:
    print(f"Error: {e}")

In [6]:
l = client.users_list()

In [ ]:
print(len(l['members']))
print("\n".join([f"{m['id']}: {m['name']}" for m in l["members"]]))

In [17]:
# 招待
try:
    client.conversations_invite(channel=channel_id, users=["U044M8GT02Y"])
except SlackApiError as e:
    print(f"Error: {e}")

Error: The request to the Slack API failed. (url: https://www.slack.com/api/conversations.invite)
The server responded with: {'ok': False, 'error': 'already_in_channel'}


In [6]:
cl = client.conversations_list()['channels']

In [ ]:
# チャンネルのアーカイブ
for c in cl:
    if c['name'].startswith("2a_"):
        print(c['id'], c['name'])
        try:
            client.conversations_join(channel=c['id'])
        except SlackApiError as e:
            print(e)
        try:
            client.conversations_archive(channel=c['id'])
        except SlackApiError as e:
            print(f"Error: {e}")

In [32]:
# 全員をチャンネルに招待するdef
import time
def invite_all_impl(channel_id):
    print(f"invite_all to {channel_id}")
    try:
        member_list = client.users_list()
    except SlackApiError as e:
        print(f"Error: {e}")
        return
    try:
        client.conversations_join(channel=channel_id)
    except SlackApiError as e:
        print(f"Error: {e}")
    for m in member_list['members']:
        time.sleep(60/50)
        print(f"invite {m['id']}: {m['name']}")
        try:
            client.conversations_invite(channel=channel_id, users=[m['id']])
        except SlackApiError as e:
            print(f"Error: {e}")


In [20]:
# チャンネル作成
channels_to_create_and_invite = [
]
channels_to_create_only = [
]

In [6]:
created = []
invited = []

In [ ]:
for c in channels_to_create_only:
    print(c)
    try:
        res = client.conversations_create(name=c)
        created.append(res["channel"])
    except SlackApiError as e:
        print(f"error: {e}")

invited = []
for c in channels_to_create_and_invite:
    print(c)
    try:
        res = client.conversations_create(name=c)
    except SlackApiError as e:
        print(f"error: {e}")
    try:
        channels = client.conversations_list(exclude_archived=True)["channels"]
        channel = [cc for cc in channels if cc["name"] == c]
        if len(channel) == 1:
            invite_all_impl(channel[0]["id"])
            invited.append(channel[0])
        else:
            print("channel not found")
    except SlackApiError as e:
        print(f"error: {e}")
        

channel_id = "C043YV4M8LC" # general
text = "以下のチャンネルを作成して全員招待しました:\n"
for c in invited:
    text += f"<#{c['id']}>\n"
text += "\n以下のチャンネルを作成しました:\n"
for c in created:
    text += f"<#{c['id']}>\n"
try:
    result = client.chat_postMessage(
        channel=channel_id,
        text=text
    )
except SlackApiError as e:
    print(f"Error: {e}")

In [16]:
channel_id = "C043YV4M8LC" # general